In [77]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pickle 

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans



## Import the data


In [78]:
df_purchase = pd.read_csv('/home/angelo/repos/vscode_repos/customer_analytics_2022/Data/purchase data.csv')

#### Describe the data in very basic terms

In [79]:
df_purchase.head()

,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,...,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
0,200000001,1,0,0,0,0,0,1.59,1.87,2.01,...,0,0,0,0,0,47,1,110866,1,0
1,200000001,11,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0
2,200000001,12,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0
3,200000001,16,0,0,0,0,0,1.52,1.89,1.98,...,0,0,0,0,0,47,1,110866,1,0
4,200000001,18,0,0,0,0,0,1.52,1.89,1.99,...,0,0,0,0,0,47,1,110866,1,0


In [80]:
# this is pointless as customers visited the store with a different number of times
df_purchase.describe()

,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,...,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
count,5.869300e+04,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,...,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000
mean,2.000003e+08,349.431074,0.249399,0.844309,0.691973,0.840799,0.248070,1.392074,1.780999,2.006789,...,0.042799,0.117851,0.035865,0.385889,0.393113,38.793962,1.101579,121841.644319,0.774198,0.655904
std,1.443167e+02,212.045058,0.432669,1.633083,1.498734,1.631628,0.431897,0.091139,0.170868,0.046867,...,0.202405,0.322434,0.185954,0.486809,0.488446,12.052447,0.652499,40643.740678,0.663247,0.794162
min,2.000000e+08,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.100000,1.260000,1.870000,...,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,38247.000000,0.000000,0.000000
25%,2.000001e+08,161.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.340000,1.580000,1.970000,...,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.000000,95541.000000,0.000000,0.000000
50%,2.000003e+08,343.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.390000,1.880000,2.010000,...,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,1.000000,117971.000000,1.000000,0.000000
75%,2.000004e+08,530.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.470000,1.890000,2.060000,...,0.000000,0.000000,0.000000,1.000000,1.000000,46.000000,1.000000,138525.000000,1.000000,1.000000
max,2.000005e+08,730.000000,1.000000,5.000000,15.000000,5.000000,1.000000,1.590000,1.900000,2.140000,...,1.000000,1.000000,1.000000,1.000000,1.000000,75.000000,3.000000,309364.000000,2.000000,2.000000


In [81]:
df_purchase.isnull().sum()

ID                   0
Day                  0
Incidence            0
Brand                0
Quantity             0
Last_Inc_Brand       0
Last_Inc_Quantity    0
Price_1              0
Price_2              0
Price_3              0
Price_4              0
Price_5              0
Promotion_1          0
Promotion_2          0
Promotion_3          0
Promotion_4          0
Promotion_5          0
Sex                  0
Marital status       0
Age                  0
Education            0
Income               0
Occupation           0
Settlement size      0
dtype: int64

## Data Segmentation

remember; in the before subject we segmented the data by following the following process:
1. standardize the data (7 original features)
2. then applied pca to reduce the 7 original features into 3 components
3. then, onto the 3 components a k-means was applied to segment the data into k = 4 groups

GOAL NOW: We got new customer data; so apply the already trained model onto them to class them accordingly 

in order for that to work:
1. select each individual customer ONCE from the data above (as one customer can appear multiple times in the dataset for different purchases)
2. then transform the customers using the already trained scaler (you use the already trained scaler as the pca and kmeans before were also build using this scaler; if you would retrain, the input might change as eg a certain range may now be larger)
3. then decompose the number of features for those new customers as well using the already trained pca model
4. after that you can apply the k means model that was already trained in order for the new customers to be grouped in one of the 4 clusters/segments previously identified; so you use the kmeans to predict the segment of the new customers based on the preexisting segments that were established using the original data

#### Import the Segmentation models

pickle models are in the same directory as the notebook initially!

In [82]:
scaler = pickle.load(open('/home/angelo/repos/vscode_repos/customer_analytics_2022/pickle_data_models/scaler.pickle', 'rb'))

In [83]:
pca = pickle.load(open('/home/angelo/repos/vscode_repos/customer_analytics_2022/pickle_data_models/pca_3.pickle', 'rb'))

In [84]:
kmeans_pca = pickle.load(open('/home/angelo/repos/vscode_repos/customer_analytics_2022/pickle_data_models/kmeans_pca_4.pickle', 'rb'))

#### apply the already trained scaler on the new data 

In [85]:
# only apply transform; fit would mean that the model is retrained; here we only want to apply it on the data --> so only transform
features = df_purchase.iloc[:, -7:]

# apply the scaler
df_purchase_segm_std = scaler.transform(features)

#### apply the already trained pca_3 (here just pca) on the data 

In [92]:
# remember that customers may be duplicates
b = pca.transform(df_purchase_segm_std)

#### apply the already trained kmeans model which was trained on the 3 components of the pca model and had k = 4 groups

In [95]:
# now apply the kmeans on it: use the predict method here
purchase_segm_kmeans_pca = kmeans_pca.predict(df_purchase_segm_pca)

#### this is now the new dataframe containing the predicted segments for the customers

In [99]:
df_purchase_predictors = df_purchase.copy()
df_purchase_predictors['Segment'] = purchase_segm_kmeans_pca

In [ ]:
# get unique customers
# note: You might  skip on this one as every instance is equally scaled anyway
df_purchase_customers = df_purchase.drop_duplicates(subset = ['ID'], keep = 'first').reset_index()

# select the relevant features (7 in number)
df_purchase_customers = df_purchase_customers.iloc[:, -7:]

# obtain the column names
feature_names = df_purchase_customers.columns.values


df_purchase_std = pd.DataFrame(data = scaler.transform(df_purchase_customers),
                    columns=features)